# CHAID

The EPC data contains several categorical variables with a lot of values. In order to find suitable features which will retain the most information, three feature sets are explored;
* data driven
* domain driven
* exhaustive

The first approach, termed data driven, uses statistical methods to reduce the number of variables. As the variables containing textual descriptions of the property have been created free-hand, many contain a large number of unique values. In some cases, only recorded for one property. The data driven approach uses a single level Chi-square Automatic Interaction Detector (CHAID) to group the levels within each categorical variable into a smaller number of groups. CHAID groups values with a similar response rate or in this context Energy Efficiency Rating (EER). 

This script run CHAID and stores the results in a dictionary

In [1]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json
from CHAID import Tree
import re

In [2]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_chaid_fname = config['DEFAULT']['epc_chaid_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [3]:
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),
                        header = 0,
                        delimiter = ',',
                        dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

C:\Users\KingRemy\AppData\Local\Temp\ipykernel_15132\425336681.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),


In [4]:
#Get quantile boundafries
quantiles = epc_train['CURRENT_ENERGY_EFFICIENCY'].describe()
print(quantiles['25%'])
print(quantiles['75%'])

56.0
73.0


In [5]:
#Create a new new target field within the training data
min_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].min()
max_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].max()
epc_train['eff_flag'] = pd.cut(epc_train['CURRENT_ENERGY_EFFICIENCY'],
                               bins = [min_eff,56.0,73.0,max_eff],
                               labels = ['0','99','1'])

#Drop unwated '99' level of eff_flag and convert to integer
epc_train = epc_train[epc_train['eff_flag'].isin(['0','1'])]
epc_train['eff_flag'] = epc_train['eff_flag'].astype(int)

In [6]:
#Get the {0,1} sample size by taking the smallest class from the 0
#and 1 outcomes
sample_size = epc_train['eff_flag'].value_counts().min()


#Subsample positive and negative samples
neg_eff_flag = epc_train[epc_train['eff_flag'] == 0].sample(sample_size,random_state = 1234,axis = 0)

pos_eff_flag = epc_train[epc_train['eff_flag'] == 1].sample(sample_size,random_state = 1234,axis = 0)

#Concatenate
epc_chaid = pd.concat([neg_eff_flag, pos_eff_flag])
epc_chaid['eff_flag'].value_counts()

#Randomly shuffle epc_CHAID and reset the index
epc_chaid = epc_chaid.sample(frac = 1).reset_index(drop=True)

In [7]:
epc_chaid

,LMK_KEY,region,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,COUNTY,...,FLOOR_HEIGHT,MECHANICAL_VENTILATION,inspection_year,floors_average_thermal_transmittance,SECONDHEAT_DESCRIPTION,LIGHTING_DESCRIPTION,low_energy_lighting_perc,roof_average_thermal_transmittance,walls_average_thermal_transmittance,eff_flag
0,993440771432014092515085604278505,Thanet,CT12 6QX,7514582178,E,45,House,Mid-Terrace,2014-09-24,Kent,...,NaN,natural,2014,NaN,"Room heaters, mains gas",low energy lighting 80% of fixed outlets,80.0,NaN,NaN,0
1,506194479962017041122155827098213,Dover,CT13 0PG,1273747768,F,27,House,Semi-Detached,2017-04-11,Kent,...,2.45,natural,2017,NaN,None,low energy lighting 60% of fixed outlets,60.0,NaN,NaN,0
2,172478398912016061417050693960352,Hyndburn,BB5 4AT,3225833568,D,55,House,Semi-Detached,2016-06-14,Lancashire,...,2.80,natural,2016,NaN,"Room heaters, mains gas",no low energy lighting,NaN,NaN,NaN,0
3,cda70534b1823b7871ebed55dace911f28edfd3fca6218...,Cherwell,OX27 9AX,10000302845,E,50,House,Semi-Detached,2021-03-17,Oxfordshire,...,2.87,natural,2021,NaN,None,low energy lighting 60% of fixed outlets,60.0,NaN,NaN,0
4,907117857732013040316124637278507,Worthing,BN13 1HW,5767576078,F,34,House,Mid-Terrace,2013-03-22,West Sussex,...,NaN,natural,2013,NaN,"Room heaters, electric",low energy lighting 50% of fixed outlets,50.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912829,1309878129742015042215345132552128,Ashford,TN26 2HS,5708225378,E,49,House,Semi-Detached,2015-04-22,Kent,...,NaN,natural,2015,NaN,None,low energy lighting 20% of fixed outlets,20.0,NaN,NaN,0
1912830,1481276359222016092216073507798726,Gedling,NG4 2FH,2052537478,F,38,House,Semi-Detached,2016-09-21,Nottinghamshire,...,2.59,natural,2016,NaN,"Room heaters, mains gas",low energy lighting 40% of fixed outlets,40.0,NaN,NaN,0
1912831,1567938489922017081812071143068243,Exeter,EX4 8QW,4483843578,F,37,House,Mid-Terrace,2017-08-16,Devon,...,NaN,natural,2017,NaN,"Room heaters, electric",low energy lighting 10% of fixed outlets,10.0,NaN,NaN,0
1912832,1661984051832018091017185523978104,Arun,PO22 7AR,5423120678,E,53,Maisonette,Detached,2018-09-10,West Sussex,...,NaN,natural,2018,NaN,Portable electric heaters (assumed),no low energy lighting,NaN,NaN,NaN,0


In [8]:
#Numeric
var_list_num = epc_chaid.select_dtypes(include= 'number').columns.tolist()
var_list_num.remove('CURRENT_ENERGY_EFFICIENCY')

#Categorical
var_list_cat = epc_chaid.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
var_list_cat.remove('POSTCODE')
var_list_cat.remove('CURRENT_ENERGY_RATING')

### Creating a dictionary of CHAID scores

In [9]:
chaid_dict = {}
for var in var_list_cat:
    #Set the inputs and outputs
    #The imputs are given as a dictionary along with the type
    #The output must be of string type
    #I have assume all features are nominal, we can change the features dictionary to include the ordinal type
    features = {var:'nominal'}
    label = 'eff_flag'
    #Create the Tree
    chaid_dict[var] = {}
    tree = Tree.from_pandas_df(epc_chaid, i_variables = features, d_variable = label, alpha_merge = 0.0)
    #Loop through all the nodes and enter into a dictionary
    print(f'\n\n\nVariable: {var}')
    print(f'p-value: {tree.tree_store[0].split.p}')
    print(f'Chi2: {tree.tree_store[0].split.score}')
    for i in range(1, len(tree.tree_store)):
        count = tree.tree_store[i].members[0] + tree.tree_store[i].members[1]
        rate = tree.tree_store[i].members[1] / count
        print(f'\nNode {i}:\n\tCount = {count}\tRate = {rate}')
        print(f'\t{tree.tree_store[i].choices}')
        chaid_dict[var]['node' + str(i)] = tree.tree_store[i].choices




Variable: region
p-value: 0.0
Chi2: 65517.82722484344

Node 1:
	Count = 433464.0	Rate = 0.45513352896665005
	['Adur', 'Gedling', 'Bassetlaw', 'Cotswold', 'Teignbridge', 'Boston', 'Thanet', 'Hastings', 'Ashfield', 'Babergh', 'Lewes', 'Waverley', 'Breckland', 'Guildford', 'Worcester', 'Fylde', 'Wealden', 'Bolsover', 'Brentwood', 'Carlisle', 'Broadland', 'Rochford', 'Dover', 'Stroud', 'Chichester']

Node 2:
	Count = 301953.0	Rate = 0.37460796878984476
	['Allerdale', 'Rossendale', 'Craven', 'Broxtowe', 'Ryedale', 'Torridge', 'Copeland', 'Richmondshire', 'Erewash', 'Tendring', 'Worthing', 'Rother', 'Scarborough', 'Hambleton', 'Harrogate', 'Lancaster', 'Wyre', 'Melton', 'Sevenoaks', 'Maldon']

Node 3:
	Count = 364964.0	Rate = 0.5059348319286285
	['Arun', 'Blaby', 'Sedgemoor', 'Braintree', 'Swale', 'Havant', 'Stafford', 'Canterbury', 'Cheltenham', 'Preston', 'Bromsgrove', 'Spelthorne', 'Chesterfield', 'Fenland', 'Mansfield', 'Elmbridge', 'Mendip', 'Rushcliffe', 'Tandridge', 'Gravesham']

N

In [10]:
%store chaid_dict

Stored 'chaid_dict' (dict)
